In [ ]:
import os
import pandas as pd
from datetime import date
start_day = date.today()
end_date = start_day
print(end_date)
search_term = '#bankofamerica'
from_date = '2021-12-01'
os.system(f"snscrape --since {from_date} twitter-search '{search_term} until:{end_date}' > result-tweets.txt")
if os.stat("result-tweets.txt").st_size == 0:
          counter = 0
else: 
          df = pd.read_csv('result-tweets.txt', names=['link'])
          counter = df.size
          
print('Number of Tweets : ' + str(counter))


In [ ]:
max_results = 5000
extracted_tweets = "snscrape --format '{content!r}'"+ f" --max-results {max_results} --since {from_date} twitter-search '{search_term} until:{end_date}' > extracted-tweets.txt"
os.system(extracted_tweets)
if os.stat("extracted-tweets.txt").st_size == 0:
    print("No tweets found")
else: 
    df = pd.read_csv('extracted-tweets.txt', names=['content'])
    for row in df['content'].iteritems():
        print(row)

In [ ]:
import wordcloud
from nltk.corpus import stopwords
import nltk
import string
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
stop = stopwords.words('english')
df['stopwords'] = df['content'].apply(lambda x: len([x for x in str(x).split() if x in stop]))
df[['content','stopwords']].head()

In [ ]:
df['content'] = df['content'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['content'].head()

In [ ]:
df['content'] = df['content'].str.replace('[^\w\s]','')
df['content'].head()

In [ ]:
df['content'] = df['content'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['content'].sample(10)

In [ ]:
def remove_url(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)
import re
import string

df['content'] = df['content'].apply(lambda x: remove_url(x))

In [ ]:
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)
df['content'] = df['content'].apply(lambda x: remove_html(x))

In [ ]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags 
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
# remove all emojis from df
df['content'] = df['content'].apply(lambda x: remove_emoji(x))



In [ ]:
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)
df['content'] = df.content.apply(round1)


In [ ]:
def clean_text_round2(text):
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)
df['content'] = df.content.apply(round2)
w = wordcloud.WordCloud(background_color="white")  

def newsfunction(a):
    res = str()
    for i in a:
        res +=','
        res += i
    return ''.join(res)

freq = pd.Series(' '.join(df['content']).split()).value_counts()[:50]
new_data = pd.DataFrame(freq)
new_data['keywords'] = freq.index
abc = newsfunction(new_data['keywords'])
w.generate(abc)
w.to_file("pywordcloud2.png")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
STOPWORDS = stopwords.words('english')
vect = TfidfVectorizer(stop_words=STOPWORDS,max_features=1000)
vect_text = vect.fit_transform(df['content'])

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=30,
learning_method='online',random_state=100,max_iter=20) 
lda_top=lda_model.fit_transform(vect_text)

In [ ]:
def maxi(x):
    maximum = x[0]
    for i in range (0,len(x)):
        if x[i] >= maximum:
            maximum = x[i]
            index=i
    return index

In [ ]:
vocab = vect.get_feature_names()
topics=[]
for i, comp in enumerate(lda_model.components_):
     vocab_comp = zip(vocab, comp)
     sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
     c, v =  zip(*sorted_words)
     topics.append(c)
     print(" ")
     print("Topic "+str(i)+": ")
     for t in sorted_words:
#             topics[i].append(t[0])
            print(t[0],end=" ")
     print("\n")

In [ ]:
topic_index=[]
for i,topic in enumerate(lda_top):
#     print("Document i: ")
    print("tweet ",i,": ",maxi(topic))
    topic_index.append(maxi(topic))

In [ ]:
df['topic_index'] = topic_index

In [ ]:
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

X=df['content'].values
y=df['topic_index'].values.reshape(-1,1)

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=123,random_state=18,max_depth=300)
forest.fit(X_train,y_train)
y_pred_forest = forest.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_forest))